## Overview

(drafting)

## Preparation

In [ ]:
# Import libraries

import sys
import os

import boto3
import sagemaker

sys.path.insert( 0, os.path.abspath( "../common/test_utility" ) )
import panorama_test_utility

In [ ]:
# Initialize variables and configurations

boto3_session = boto3.session.Session()
sm_session = sagemaker.Session()

account_id = boto3.client("sts").get_caller_identity()["Account"]
region = boto3_session.region_name
s3_bucket = sm_session.default_bucket()
sm_role = sagemaker.get_execution_role()

print( "account_id :", account_id )
print( "region :", region )
print( "s3_bucket :", s3_bucket )
print( "sm_role :", sm_role )

In [ ]:
app_name = "lab1"
code_package_name = f"{app_name}_code"
code_package_version = "1.0"

## Create a simplest Panorama application - "Hello World!"

In [ ]:
!panorama-cli init-project --name {app_name}

In [ ]:
!cd {app_name} && panorama-cli create-package --name {code_package_name} --type Container --version {code_package_version}

#### Manually create/edit application source code, code package descriptor

1. Create a Python source code "lab1/packages/357984623133-lab1_code-1.0/src/app.py" with following single line of code (You can copy & paste from "app-v1.py"):

    ```
    print("Hello World!")
    ```


2. Edit "lab1/packages/357984623133-lab1_code-1.0/descriptor.json", and replace <entry_file_name_under_src> with "app.py"

    ```
    "name": "/panorama/app.py"
    ```


In [ ]:
source_filename = f"./{app_name}/packages/{account_id}-{code_package_name}-{code_package_version}/src/app.py"

print("Source file:", source_filename)
assert os.path.exists(source_filename)

#### Build container image (Test Utility doesn't use the container image itself, but uses graph information)

In [ ]:
!cd {app_name} && panorama-cli build-container --container-asset-name code --package-path packages/{account_id}-{code_package_name}-{code_package_version}

## Run the Hello World application with "Test Utility"

In [ ]:
%run ../common/test_utility/panorama_test_utility_run.py \
\
--app-name {app_name} \
--code-package-name {code_package_name} \
--py-file {source_filename}


## Simple video pass-through app

#### Manually edit the application source code

1. Edit "lab1/packages/357984623133-lab1_code-1.0/src/app.py". You can copy & paste from "app-v2.py".


#### Add camera input node and HDMI output node

In [ ]:
camera_node_name = f"{app_name}_camera"
data_sink_node_name = f"{app_name}_data_sink"

In [ ]:
!cd {app_name} && panorama-cli add-panorama-package --type camera --name {camera_node_name}

In [ ]:
!cd {app_name} && panorama-cli add-panorama-package --type data_sink --name {data_sink_node_name}

#### Manually edit the graph.json and connect nodes

1. Open "lab1/graphs/lab1/graph.json" by text editor.
2. Add edges to connect camera -> code, and code -> hdmi.

    ```
        "edges": [
            {
                "producer": "lab1_camera.video_out",
                "consumer": "code_node.video_in"
            },
            {
                "producer": "code_node.video_out",
                "consumer": "lab1_data_sink.video_in"
            }            
        ]
    ```


#### Prepare a video file to simulate camera input

In [ ]:
# !wget xyz

os.makedirs( "videos", exist_ok=True )
!cp ../../../aws-panorama-samples/samples/common/test_utility/videos/TownCentreXVID.avi ./videos/

## Run the Video pass-through application with "Test Utility"

In [ ]:
%run ../common/test_utility/panorama_test_utility_run.py \
\
--app-name {app_name} \
--code-package-name {code_package_name} \
--py-file {source_filename} \
\
--camera-node-name lab1_camera \
--video-file ./videos/TownCentreXVID.avi \
--video-start 0 \
--video-stop 10 \
--video-step 1 \
\
--output-screenshots ./screenshots/%Y%m%d_%H%M%S

#### Check generated screenshots

1. Browse ./screenshots/{date_time} in File browser pane.
2. Open one of screenshot file.

## Extend the pass-through application to real Computer Vision application

#### Export "yolo3_mobilenet1.0_coco" from Gluoncv model zoo

In [ ]:
import gluoncv
import tarfile

In [ ]:
# Initialize model related variables and configurations

model_package_name = f"{app_name}_model"
model_package_version = "1.0"

people_detection_model_name = "people_detection_model"

In [ ]:
def export_model_and_create_targz( prefix, name, model ):
    os.makedirs( prefix, exist_ok=True )
    gluoncv.utils.export_block( os.path.join( prefix, name ), model, preprocess=False, layout="CHW" )

    tar_gz_filename = f"{prefix}/{name}.tar.gz"
    with tarfile.open( tar_gz_filename, "w:gz" ) as tgz:
        tgz.add( f"{prefix}/{name}-symbol.json", f"{name}-symbol.json" )
        tgz.add( f"{prefix}/{name}-0000.params", f"{name}-0000.params" )
        
    print( f"Exported : {tar_gz_filename}" )

In [ ]:
# Export object detection model. Reset the classes for human detection only.
people_detection_model = gluoncv.model_zoo.get_model('yolo3_mobilenet1.0_coco', pretrained=True)
people_detection_model.reset_class(["person"], reuse_weights=['person'])
export_model_and_create_targz( "models", "yolo3_mobilenet1.0_coco_person", people_detection_model )


#### Add exported model files in the model package

In [ ]:
!cd {app_name} && panorama-cli create-package --name {model_package_name} --type Model --version {code_package_version}

#### Manually edit model descriptor file

Edit "lab1/packages/357984623133-lab1_model-1.0/descriptor.py" with following values:

```
"framework": "MXNET",
"inputs": [
    {
        "name": "data",
        "shape": [
            1, 3, 480, 600
        ]
    }
]
```


#### Add the model in the model package

In [ ]:
!cd {app_name} && panorama-cli add-raw-model \
    --model-asset-name {people_detection_model_name} \
    --model-local-path ../models/yolo3_mobilenet1.0_coco_person.tar.gz \
    --descriptor-path packages/{account_id}-{model_package_name}-{model_package_version}/descriptor.json \
    --packages-path packages/{account_id}-{model_package_name}-{model_package_version}

#### Compile the model to locally run with Test Utility

In [ ]:
people_detection_model_data_shape = '{"data":[1,3,480,600]}'

%run ../common/test_utility/panorama_test_utility_compile.py \
\
--s3-model-location s3://{s3_bucket}/panorama-workshop/{app_name} \
\
--model-node-name {people_detection_model_name} \
--model-file-basename ./models/yolo3_mobilenet1.0_coco_person \
--model-data-shape '{people_detection_model_data_shape}' \
--model-framework MXNET

#### Manually edit the application source code

1. Edit the source code "lab1/packages/357984623133-lab1_code-1.0/src/app.py" referring to "app-v3.py":


## Run the People detection application with "Test Utility"

In [ ]:
video_filepath = "videos/TownCentreXVID.avi"

%run ../common/test_utility/panorama_test_utility_run.py \
\
--app-name {app_name} \
--code-package-name {code_package_name} \
--py-file {source_filename} \
\
--model-package-name {model_package_name} \
--model-node-name {people_detection_model_name} \
--model-file-basename ./models/yolo3_mobilenet1.0_coco_person \
\
--camera-node-name lab1_camera \
\
--video-file ./videos/TownCentreXVID.avi \
--video-start 0 \
--video-stop 10 \
--video-step 1 \
\
--output-screenshots ./screenshots/%Y%m%d_%H%M%S
